In [3]:
from datetime import datetime, timedelta
import pandas as pd
from google.cloud import storage
import yfinance as yf
import io



ImportError: cannot import name 'storage' from 'google.cloud' (unknown location)

In [6]:
ls

 O volume na unidade C � Acer
 O N�mero de S�rie do Volume � DAF1-E8C5

 Pasta de c:\Users\andre\Documents\_Projects\_Automated Financial Intelligence System

07/02/2026  21:52    <DIR>          .
07/02/2026  18:49    <DIR>          ..
07/02/2026  23:45    <DIR>          AutoFinIntSys
               0 arquivo(s)              0 bytes
               3 pasta(s)   38.942.097.408 bytes dispon�veis


In [5]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
